In [1]:
import os
import pandas as pd
import re

os. chdir("c:\\Users\\simon\\OneDrive\\Dokumente\\UNILU\\3 - HS24\\1 MA\\business-reports-nlp")

### import texts from txt files in folders into dataframe

In [14]:
def read_text_files_in_folder(folder_path):
    data = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".txt"):
                full_path = os.path.join(root, file)
                with open(full_path, 'r', encoding='utf-8') as f:
                    content = f.read()
                
                # Extract the immediate parent directory (subfolder) of the file
                subfolder_name = os.path.basename(os.path.dirname(full_path))
                
                data.append({
                    'file_name': file,
                    'file_path': full_path,
                    'source': subfolder_name,
                    'content': content.replace('\n', ' ')
                })
                
    return pd.DataFrame(data)

# Example usage
folder_path = "texts/txts/tagged/_final"
df = read_text_files_in_folder(folder_path)


### split texts according to \<break\> characters

In [15]:
# Initialize a list to store the new rows
new_rows = []

# Iterate over each row in the original DataFrame
for index, row in df.iterrows():
    # Split the content into sentences based on '.', '!', '?'
    sentences = re.split(r'<break>', row['content'])
    
    # Filter out empty strings from the list of sentences
    sentences = [s.strip() for s in sentences if s.strip()]
    
    # Create a new row for each sentence and append it to the list
    for sentence in sentences:
        new_row = {
            'file_name': row['file_name'],
            'file_path': row['file_path'],
            'source': row['source'],
            'content': sentence
        }
        new_rows.append(new_row)

# Create a new DataFrame from the list of new rows
texts = pd.DataFrame(new_rows)

In [16]:
sum(texts['content'].str.count('<MIA>'))

23

In [17]:
texts = texts[~texts['content'].str.contains('<MIA>')]

In [18]:
texts['year'] = texts['file_name'].str[:4]
texts['year'] = texts['year'].astype(int)

In [19]:
texts = texts[texts['content'].str.count(' ') >= 10]

In [20]:
texts.to_csv('data/paragraphs.csv', index = False, encoding='utf-8')

## Export csv to excel

In [3]:
texts = pd.read_csv('data/paragraphs.csv', encoding = 'utf-8')

In [8]:
def remove_illegal_characters(s):
    return ''.join([char for char in s if char.isprintable()])

# Apply the function to each string column
for col in texts.select_dtypes(include=['object']):
    texts[col] = texts[col].apply(remove_illegal_characters)

texts.to_excel('data/paragraphs.xlsx', index=False, engine='openpyxl')

## Randomly create pre-selection

In [18]:
texts = pd.read_csv('data/paragraphs.csv', encoding = 'utf-8')

In [19]:
sample = texts.sample(n=250)

In [20]:
def remove_illegal_characters(s):
    return ''.join([char for char in s if char.isprintable()])

# Apply the function to each string column
for col in sample.select_dtypes(include=['object']):
    sample[col] = sample[col].apply(remove_illegal_characters)

sample.to_excel('data/sample.xlsx', index=False, engine='openpyxl')

### Sample another 1250 cases

In [21]:
sample = pd.read_excel('data/sample.xlsx')
sample = sample.drop(['index', 'category', 'key', 'note'], axis=1)

In [22]:
# Replace 'common_column' with the actual name of the column to compare
common_column = 'content'

# Merge the two DataFrames based on the common column
merged_df = texts.merge(sample, on=common_column, how='left', indicator=True)

# Keep only the rows that are in df1 but not in df2 based on the indicator column
filtered_df1 = merged_df[merged_df['_merge'] == 'left_only']

# Drop the indicator column
filtered_df1 = filtered_df1.drop(columns=['_merge'])

# Now, filtered_df1 contains the rows from df1 where the 'common_column' values
# are not identical to the values in df2's 'common_column'.

new_sample = filtered_df1.sample(n=1250, random_state=42)

In [23]:
if sample['content'].isin(new_sample['content']).any():
    print("There are identical entries between df1 and df2.")
else:
    print("There are no identical entries between df1 and df2.")

There are no identical entries between df1 and df2.


In [26]:
def remove_illegal_characters(s):
    if isinstance(s, str):
        return ''.join([char for char in s if char.isprintable()])
    else:
        return s

# Apply the function to each string column
for col in new_sample.select_dtypes(include=['object']):
    new_sample[col] = new_sample[col].apply(remove_illegal_characters)

new_sample.to_excel('data/sample2.xlsx', index=False, engine='openpyxl')